In [1]:
##TODO: Randomize seed / seasonize seed (don't base it off last one - overconverges)

In [2]:
# default_exp core

# jetson-haiku

> API details.


In [3]:
# -*- coding: utf-8 -*-

"""Top-level package for jetson-haiku, GPT2 haikubot for Jetson Nano"""

__author__ = """LemurTime"""
__email__ = "software@ananthropicprose.com"
__version__ = "0.0.2"

#0.0.2: added bad word list, filter for invalid characters, seasonal haiku seed, datestamp and haiku seed to log.  

#Todo: Update with new GPT2 model
#Todo: fine-tine GPT2 output
#Todo: add Twitter output?

In [4]:
#export

#define list of all haikus
global finished_haiku
global finished_haiku_list
#global GPT2_seed_text
verse_one_string=""
verse_two_string=""
verse_three_string=""
haiku_seed=""

In [5]:
#export
import numpy as np
import syllapy
import re
import random
#import gpt2Pytorch as gp2py
#rather, let's just incorporate the gp2pytorch code, for now:

In [6]:
#export

#Initial arguments go here:

#GPT2_seed_text="Cherry trees in the summer."
args_nsamples = 1
args_batch_size = -1
args_length = 1
args_unconditional = 0
args_temperature = 0.9
args_top_k = 40
args_quiet = 1
verse_input = ""



In [7]:
#export

#Rather than import GPT2, code here (need to update with new GPT2 model later) 
#Need to fix this, and import rather than keep GPT2 directory in folder

'''
    code by TaeHwan Jung(@graykode)
    Original Paper and repository here : https://github.com/openai/gpt-2
    GPT2 Pytorch Model : https://github.com/huggingface/pytorch-pretrained-BERT
'''
import os
import sys
import torch
import random
import argparse
import numpy as np
from GPT2.model import (GPT2LMHeadModel)
from GPT2.utils import load_weight
from GPT2.config import GPT2Config
from GPT2.sample import sample_sequence
from GPT2.encoder import get_encoder

def text_generator(state_dict):
   # parser = argparse.ArgumentParser()
  #  parser.add_argument("--text", type=str, required=True)
   # parser.add_argument("--quiet", type=bool, default=False)
   # parser.add_argument("--nsamples", type=int, default=1)
   # parser.add_argument('--unconditional', action='store_true', help='If true, unconditional generation.')
   # parser.add_argument("--batch_size", type=int, default=-1)
   # parser.add_argument("--length", type=int, default=-1)
   # parser.add_argument("--temperature", type=float, default=0.7)
   # parser.add_argument("--top_k", type=int, default=40)
   # args = parser.parse_args()

    if args_quiet is False:
        print(args)

   # if args_batch_size == -1:
    args_batch_size = 1
    assert args_nsamples % args_batch_size == 0

    seed = random.randint(0, 2147483647)
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load Model
    enc = get_encoder()
    config = GPT2Config()
    model = GPT2LMHeadModel(config)
    model = load_weight(model, state_dict)
    model.to(device)
    model.eval()

    #if args_length == -1:
    args_length = config.n_ctx // 2
   # elif args_length > config.n_ctx:
    #    raise ValueError("Can't get samples longer than window size: %s" % config.n_ctx)

   # print(args.text)
    context_tokens = enc.encode(GPT2_seed_text)

    generated = 0
    for _ in range(args_nsamples // args_batch_size):
        out = sample_sequence(
            model=model, length=args_length,
            context=context_tokens  if not  args_unconditional else None,
            start_token=enc.encoder['<|endoftext|>'] if args_unconditional else None,
            batch_size=args_batch_size,
            temperature=args_temperature, top_k=args_top_k, device=device
        )
        out = out[:, len(context_tokens):].tolist()
        for i in range(args_batch_size):
            generated += 1
            text = enc.decode(out[i])
            if args_quiet is False:
                print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
            global GPT2_output
            GPT2_output = text
            print(text)

if __name__ == '__main__':
    if os.path.exists('gpt2-pytorch_model.bin'):
        state_dict = torch.load('gpt2-pytorch_model.bin', map_location='cpu' if not torch.cuda.is_available() else None)
     #   text_generator(state_dict)
    else:
        print('Please download gpt2-pytorch_model.bin')
        sys.exit()


In [8]:
#export


##UPDATING VERSE GEN CODE TO ADD WORD FILTER - REMOVE ARTICLES AT END OF LINE 
##TODO - REMOVE GLOBAL VARIABLES
##TODO - add Kigo / seasonal word

##General verse_gen - input is input, number of syllables required

def verse_gen(verse_input, syllable_length):
    global verse_words
    global verse_string
    global verse_count
    global verse_syllable_count
    
#Go to first whitespace, count syllables.  Continue until "syllable_length" syllables.  If over required amount syllables try with new input.
#initialize counters

    #left side location in string
    y=0
    
    #right side location in string
    x=1
    
    #counter for number of times to try this before getting new input
    z=0
    
    #initial syllable count of zero
    verse_syllable_count=0

    #Split the input to make it a list of words
    verse_words=verse_input.split(' ')

#Set a counter - try this a number of times then move on
    while z < 40:
    
        if verse_syllable_count < syllable_length:
            print("Adding next word to the string")
        
            #Put words from y to x in a string
            verse_string=' '.join(verse_words[y:x])

                
            #If no special characters, Count the syllables
            #Validate haiku-able phrases.  If not, reinitialize the string and keep going
            
            #Get rid of any phrase with numerals or special characters in it
            
            if re.match("^[a-zA-Z\s-]*$", verse_string):
                verse_syllable_count = syllapy.count(verse_string)
    
                #increment x
                x=x+1

                #increment z
                z=z+1

                #print the string
                print(verse_string)
            
            else:
                print(verse_string, "Invalid characters. Moving up in string.")
                #reinitialize verse_string
                verse_string=""
                verse_syllable_count=0
                y=y+1
                
            
        if verse_syllable_count == syllable_length:
            #Get rid of any trailing spaces
            verse_string = verse_string.strip()
            last_word=verse_string.split(" ")[-1]
            #Get rid of any phrase ending in an article or preposition words
            
            bad_word_list = 'as of the and if or at by our your my'
            bad_word_set = set(bad_word_list.split())

            if (last_word in bad_word_set):
                
                print(verse_string, ": String ends in article or other undesirable word.  Moving up in string")
                
                #reinitialize verse_string
                verse_string=""
                verse_syllable_count=0
                y=y+1
           
            #If successful, print string and return    
            else:
                print("Haiku line found:\n", verse_string)
                return verse_string
    
        if verse_syllable_count > syllable_length:
            #If string is too long, reinitialize the string and keep going

            print(verse_string, ": String too long. Moving up in string.")
            
            #reinitialize verse_string
            verse_string=""
            verse_syllable_count=0
            y=y+1
            
            #verse_gen(verse_input, syllable_length)

    #Get a new string and start over if this string doesn't work
    if z == 40:
        print("Getting new input.")
        text_generator(state_dict)
        verse_input = GPT2_output
        verse_string=verse_gen(verse_input, syllable_length)
        return verse_string

#END OF VERSE ONE GEN    

In [9]:
#export

## Code to run the module

def haiku_gen():
    import random
    global GPT2_seed_text
    global haiku_seed
    lines = open('sajiki.txt').read().splitlines()
    GPT2_seed_text = random.choice(lines)
    haiku_seed = GPT2_seed_text
   
    text_generator(state_dict)

#Code to generate verse 1:
    verse_string = ""
    verse_input = GPT2_output
    syllable_length = 5
    verse_one_string=verse_gen(verse_input, syllable_length)
    
#Code to generate verse 2:
    verse_string = ""
    GPT2_seed_text = verse_one_string
    text_generator(state_dict)
    verse_input = GPT2_output
    syllable_length = 7
    verse_two_string=verse_gen(verse_input, syllable_length)

#Code to generate verse 3:
    verse_string = ""
    GPT2_seed_text = verse_one_string
    text_generator(state_dict)
    verse_input = GPT2_output
    syllable_length=5
    verse_three_string=verse_gen(verse_input, syllable_length)

#Print finished haiku
    print("Here is the haiku seed:")
    print(haiku_seed)
    print("Here is the haiku:")

#Print finished haiku
 
    finished_haiku=''
    finished_haiku='\n'.join([verse_one_string,verse_two_string,verse_three_string])
    print(finished_haiku)
    
    #Add finished haiku to a list and date
    #Todo add date and time
     #import datetime
    from datetime import datetime

    # get current date
    now = datetime.now()
    
    # convert current date into timestamp
    haiku_time = now.strftime("%m/%d/%Y, %H:%M:%S")

    f = open("haikulist.txt", "a")
    f.write("\n\nHaiku seed:\n"+haiku_seed+"\n\nHaiku:\n"+finished_haiku+"\n"+haiku_time)
    f.close()
    
    #Place finished haiku in an input for GUI (clear it out first)
    f = open("latesthaiku.txt", "w")
    f.write(finished_haiku)
    f.close()


    #Put verse2 in as GPT seedtext seed  ## This leads to too much convergence
    
  #  f.write(verse_two_string)
  #  get sajiki from http://jti.lib.virginia.edu/japanese/haiku/saijiki/full.html  -> sajiki.txt
   
    
    import random
    
    from IPython.display import Audio

    wave = np.sin(2*np.pi*400*np.arange(10000*2)/10000)
    Audio(wave, rate=30000, autoplay=True)
    
 


In [ ]:
#export

##Run the module:

#Initial arguments go here:
from IPython.display import Audio
#GPT2_seed_text="Gorillas in the mist."
args_nsamples = 1
args_batch_size = -1
args_length = 1
args_unconditional = 0
args_temperature = 0.9
args_top_k = 40
args_quiet = 1
verse_input = ""
z = 0

while z < 100:
   # lines = open('sajiki.txt').read().splitlines()
   # GPT2_seed_text = random.choice(lines)
   
    haiku_gen()
    
    
    #Beep after each iteration
    wave = np.sin(2*np.pi*400*np.arange(10000*2)/10000)
    Audio(wave, rate=30000, autoplay=True)
    z+1
    
#Beep when all done

wave = np.sin(2*np.pi*400*np.arange(10000*2)/10000)
Audio(wave, rate=40000, autoplay=True)

100%|██████████| 512/512 [00:46<00:00, 11.11it/s]


 1 finale, which saw the season premiere in April, is expected to return to NBC this fall.

In Season 2, NBC is hoping for a new beginning for its series with Season 1 premiere this fall. The show's lead in the series — Amy Poehler — will return in the new season. Poehler will join the cast of Parks and Recreation.

Also read: 'Parks and Recreation': 'Better Call Saul' Cast Will Come Home to 'Better Call Saul'

On Wednesday, Netflix announced the first Season 2 episode of Season 1 of Parks and Rec, which opened on The CW on Dec. 4. The new Season 2 episode opens with a shot of Poehler, Amy Poehler and Kristen Wiig in the lobby of the Beverly Hilton. The other new new show for the network is called 'The Big Bang Theory,' which opens on Jan. 24.

Also read: 'Parks and Rec' Is the Best of the Week

Read or Share this story: http://usat.ly/2eU3LhX<|endoftext|>I'm sure some of you have experienced the joys of owning something by its very nature, and when you finally do, I wish you the best 

100%|██████████| 512/512 [00:45<00:00, 11.21it/s]


 premiere of The Mummy on February 15, 2017, only the second season since 1997-1998 of Peter Lorre's The Mummy, which had been in preproduction for about three years.)

With a few new additions, the series is currently on hiatus, while the third season is in a completely different place, with the third season filming in New Zealand. As the series currently doesn't have the opportunity to officially conclude, the producers of The Mummy are still considering the next season, and are working on the possibility of returning the series to the US, if they'd like to pursue.<|endoftext|>If you need help with a problem with this blog, just share it and I'll help you solve it. And if you don't have a solution like this, but would like to try using this method as a regular blog, my free eBook is the solution!

It is a simple and practical solution.

This method is also used on blogs, websites, and other websites. The method provides a template for creating an image (it is called a template with a

100%|██████████| 512/512 [00:45<00:00, 11.16it/s]


...

100%|██████████| 512/512 [00:44<00:00, 11.60it/s]


 regions during the early days of Islam. He says that there was no official recognition of the name 'Sadda' in the Quran until now and it is more likely that this is still true today.

"It is the most common name of the Prophet. For example, it was not given to him on the day of his death when he became king. But it was later added by another person (and later became the name of Muhammad) in the history of Islam. These name was changed as a way of displaying unity in his lineage and even the Muslim world," says Malik.

This is because in addition to being related to Salami's name, it was also written on the back (the top) shoulder and on his chest, the 'Sadda' being more commonly associated with Mohammed, but not with this name.

"This means that, if you remember Salami's name, how did he choose it from his family? He was given a great name while he was king but now as the name 'Sadda' he is known as the first person to choose it. The name was changed by his father's successor on his b

100%|██████████| 512/512 [00:44<00:00, 11.58it/s]


. A.D. 730.

3.

A.D. 741.

It is now on the thirteenth day of the month of Nechaia (Mysia, or Septuagint), and the festival is celebrated at Pithae, where its celebrated poet, who was in it at the time of his decease, is described as being called in Greek by the chief priests of the house of St. Cyril to the city of Pithae-Trisius, to whom he has since devoted himself up to the age of ten years. The chief priests of Pithae at that time were also named by the most distinguished writers and poets, and they, to a small but considerable extent, were also esteemed by others of the city as the most worthy, and to the most eminent members of their school, of which no other city, at any time, ever existed. For so in other respects it is very important that any one be the object, who is to be remembered among the most eminent members of the church who were in this city at that time. In the same place it is said that as soon as the day of the festival has passed from the same hour, the people o

 81%|████████▏ | 416/512 [00:35<00:08, 11.97it/s]

In [ ]:
#hide
#from nbdev.showdoc import *

In [ ]:
#from nbdev.export import *
#notebook2script()

In [ ]:
#Todo:
#Training feature - what is a good/bad haiku


In [ ]:
   
##Todo: make ananthropic
#Remove: wordlist indicating persons or personification (I, his, hers, mine, ours, who)

In [ ]:
#Testing
finished_haiku=''

In [ ]:
verse_one_string

In [ ]:
#     import random
#     lines = open('sajiki.txt').read().splitlines()
#     myline =random.choice(lines)
#     print(myline)


In [ ]:
myline

In [ ]:
#test regex
word = "Welcome "
import re
print("Valid") if re.match("^[a-zA-Z\s]*$", word) else "Invalid"

In [ ]:
word

In [ ]:
word.strip()

In [ ]:
word

In [ ]:
verse_string="think it would go as far as plant"

In [ ]:
verse_string[-1]

In [ ]:
verse_string=verse_string.strip()

In [ ]:
verse_string[-1]

In [ ]:
last_word=verse_string.split(" ")[-1]

In [ ]:
last_word

In [ ]:
if any(word in 'as of the' for word in last_word):
    print("bad word detected")
else:
    print("OK")

In [ ]:
long_word_list = 'some one long two phrase three about above along after against plant'
long_word_set = set(long_word_list.split())
set('word along river'.split()) & long_word_set
set(['along'])

In [ ]:
if (last_word in long_word_set):
    print("WORD FOUND") 
    
else:
    print("Word not found")

In [ ]:
bad_word_list = 'as of the and if or'
bad_word_set = set(bad_word_list.split())

if (last_word in long_word_set):
    print("WORD FOUND") 
    
else:
    print("Word not found")


In [ ]:

    from datetime import datetime

# get current date
    now = datetime.now()
    
    # convert current date into timestamp
    timestamp = datetime.timestamp(now)



In [ ]:
timestamp


In [ ]:
print(timestamp)

In [ ]:
print(now)

In [ ]:
haikutime=""

In [ ]:
haikutime=now

In [ ]:
haikutime


In [ ]:
dt_object = datetime.fromtimestamp(timestamp)


In [ ]:
dt_object

In [ ]:
print(dt_object)

In [ ]:
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
print("date and time:",date_time)

In [ ]:
date_time
